In [ ]:
import cv2
import mediapipe as mp
import csv
import pandas as pd
import os

In [ ]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [157]:
DATASET_DIR = 'Single_person_violent'
CLASSES_LIST = ["Kicking","Punching"]
OUTPUT_DIR = 'Output'

In [153]:
def write_landmarks_to_csv(landmarks, frame_number, csv_data):
    #print(f"Landmark coordinates for frame {frame_number}:")
    for idx, landmark in enumerate(landmarks):
        #print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([frame_number, landmark.x, landmark.y, landmark.z, landmark.visibility])    

In [154]:
def convert_video_to_landmark_csv(video_path):
    frame_number = 0
    csv_data = []

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            frame_number += 1
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Perform pose detection
            results = pose.process(image)

            # Convert the image back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw the pose annotation on the image
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Add the landmark coordinates to the list and print them
            write_landmarks_to_csv(results.pose_landmarks.landmark, frame_number, csv_data)   
    cap.release() 
    # Write the 2D array to a CSV file
    file_name = video_path.split('\\')[-1][:-4]
    class_name = video_path.split('\\')[-2]    
    output_file = os.path.join(OUTPUT_DIR, class_name,file_name)    
    with open(f'{output_file}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write each row of the 2D array
        for row in csv_data:
            writer.writerow(row)

In [ ]:
def create_dataset(CLASSES_LIST):
    print('List: ',CLASSES_LIST)
    for class_name in CLASSES_LIST:
        print(f"Extracting data from {class_name}")
        # Get list of videos for each class
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            convert_video_to_landmark_csv(video_file_path)

In [ ]:
create_dataset(CLASSES_LIST)

In [ ]:
custom_headers = ['Frame Number', 'x','y','z','Visibility']
df = pd.read_csv('output.csv', header=None, names=custom_headers)

In [ ]:
df.head()

In [181]:
def createSequence(csv_path,label):
    custom_headers = ['Frame Number', 'x', 'y', 'z', 'Visibility']
    data = pd.read_csv(csv_path, header=None, names=custom_headers)
    #frame = [[d[1]['x'], d[1]['y'], d[1]['z'], d[1]['Visibility']] for d in data.iterrows()]
    current_frame = -1
    frame = []
    sequence = []
    sequence.append(label)
    for d in data.iterrows():
        if not current_frame == d[1]['Frame Number']:
            current_frame = d[1]['Frame Number']
            if frame:              
                sequence.append(frame)                            
                frame = []                            
        frame.append([d[1]['x'], d[1]['y'], d[1]['z'], d[1]['Visibility']])    
    
    sequence.append(frame)            
    return sequence


In [194]:
# Read data from CSV and append to processed_df
sequence = createSequence('output.csv','Kicking')
processed_df = pd.DataFrame([sequence])
#processed_df2 = processed_df.append(pd.DataFrame(frame, columns=['x', 'y', 'z', 'Visibility']), ignore_index=True)
processed_df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,Kicking,"[[0.4208270311355591, 0.2813926935195923, -0.5...","[[0.4226616024971008, 0.2830023467540741, -0.4...","[[0.422335147857666, 0.2832168936729431, -0.48...","[[0.4239648282527923, 0.2812423706054687, -0.2...","[[0.4259682595729828, 0.2790455818176269, -0.2...","[[0.424310564994812, 0.2764437794685364, -0.26...","[[0.424225777387619, 0.2755632400512695, 0.049...","[[0.4241669476032257, 0.2759043276309967, -0.1...","[[0.4190621674060821, 0.2758240699768066, -0.0...",...,"[[0.4902646839618683, 0.266271561384201, -0.52...","[[0.4924321174621582, 0.2656450271606445, -0.5...","[[0.4942595660686493, 0.2644723653793335, -0.5...","[[0.4932672381401062, 0.2639179527759552, -0.5...","[[0.4929601550102234, 0.264041006565094, -0.53...","[[0.4959222972393036, 0.2640158534049988, -0.5...","[[0.5004463791847229, 0.2640080451965332, -0.5...","[[0.5007750391960144, 0.2638412415981293, -0.4...","[[0.4997018575668335, 0.2627485990524292, -0.4...","[[0.4992328286170959, 0.262261152267456, -0.44..."


In [183]:
def createDataframe():
    df_array = []    
    for class_name in CLASSES_LIST:        

        # Get list of csv files for each class
        files_list = os.listdir(os.path.join(OUTPUT_DIR, class_name))
        for file_name in files_list:
            # Get the complete csv path.
            csv_file_path = os.path.join(OUTPUT_DIR, class_name, file_name)            
            df_array.append(createSequence(csv_file_path,class_name))
    return pd.DataFrame(df_array)
            

In [184]:
complete_df = createDataframe()

In [190]:
complete_df.columns

RangeIndex(start=0, stop=223, step=1)

In [192]:
complete_df.head()

,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
0,Kicking,"[[0.6677398681640625, 0.3022188544273376, -0.7...","[[0.6677064299583435, 0.3031054735183716, -0.7...","[[0.6696239113807678, 0.3042094111442566, -0.7...","[[0.6722891926765442, 0.305832952260971, -0.75...","[[0.6741838455200195, 0.3086952865123749, -0.7...","[[0.67625492811203, 0.3130343556404114, -0.862...","[[0.6758496761322021, 0.3170639872550964, -0.8...","[[0.6746688485145569, 0.3211084008216858, -0.9...","[[0.6721891760826111, 0.329367458820343, -0.98...",...,None,None,None,None,None,None,None,None,None,None
1,Kicking,"[[0.419850081205368, 0.2924816608428955, -0.32...","[[0.4198570549488067, 0.2927826046943664, -0.2...","[[0.4196742177009582, 0.2939316928386688, -0.2...","[[0.4194415509700775, 0.2957449555397033, -0.2...","[[0.4194997251033783, 0.2958241701126098, -0.2...","[[0.4207811653614044, 0.2957587838172912, -0.3...","[[0.4214946329593658, 0.2957068085670471, -0.3...","[[0.4235286712646484, 0.2952330112457275, -0.3...","[[0.4256711006164551, 0.2941022217273712, -0.3...",...,None,None,None,None,None,None,None,None,None,None
2,Kicking,"[[0.5217735767364502, 0.2557693421840668, -0.6...","[[0.5198756456375122, 0.253849983215332, -0.62...","[[0.5160553455352783, 0.2507854104042053, -0.6...","[[0.5162323117256165, 0.2496377229690551, -0.5...","[[0.5206785798072815, 0.2485862076282501, -0.5...","[[0.5335404276847839, 0.2485955953598022, -0.5...","[[0.5469369888305664, 0.2489242553710937, -0.6...","[[0.561162531375885, 0.255070149898529, -0.610...","[[0.569862425327301, 0.2608897387981415, -0.61...",...,None,None,None,None,None,None,None,None,None,None
3,Kicking,"[[0.470816969871521, 0.2779606282711029, -0.62...","[[0.4704335629940033, 0.2777650952339172, -0.6...","[[0.4668804109096527, 0.2772256135940552, -0.5...","[[0.4653769731521606, 0.2770519852638244, -0.5...","[[0.4617950022220611, 0.275784969329834, -0.46...","[[0.456878662109375, 0.2719708085060119, -0.43...","[[0.4521282315254211, 0.2699009478092193, -0.4...","[[0.4462897479534149, 0.2687931656837463, -0.4...","[[0.4419589638710022, 0.26866215467453, -0.441...",...,None,None,None,None,None,None,None,None,None,None
4,Kicking,"[[0.8110488653182983, 0.2549797296524048, -0.5...","[[0.8110454082489014, 0.2532932162284851, -0.3...","[[0.8074595332145691, 0.2395376414060592, -0.4...","[[0.7956667542457581, 0.2290425300598144, -0.5...","[[0.7943984866142273, 0.2215637266635894, -0.5...","[[0.7865209579467773, 0.2177729606628418, -0.6...","[[0.7721162438392639, 0.2176922261714935, -0.5...","[[0.7580470442771912, 0.2204321175813675, -0.7...","[[0.734661340713501, 0.2223648279905319, -0.73...",...,None,None,None,None,None,None,None,None,None,None


In [193]:
complete_df[complete_df.iloc[:,0] == 'Punching']

,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
21,Punching,"[[0.7002928256988525, 0.2674192488193512, -0.3...","[[0.7011668682098389, 0.2666239440441131, -0.3...","[[0.7018589377403259, 0.2653409242630005, -0.3...","[[0.703841507434845, 0.2642578482627868, -0.49...","[[0.708078920841217, 0.2640006840229034, -0.53...","[[0.7162479758262634, 0.2641276717185974, -0.5...","[[0.7233095765113831, 0.264122873544693, -0.67...","[[0.7300415635108948, 0.263948380947113, -0.65...","[[0.7326869368553162, 0.2636285722255707, -0.6...",...,None,None,None,None,None,None,None,None,None,None
22,Punching,"[[0.4576251208782196, 0.2688536047935486, 0.02...","[[0.4577584564685821, 0.2740941047668457, 0.02...","[[0.4598338305950165, 0.2804973125457763, 0.01...","[[0.4639894366264343, 0.2888439297676086, 0.01...","[[0.4662812948226928, 0.295228511095047, 0.006...","[[0.4679256379604339, 0.2947286367416382, 0.10...","[[0.4644545316696167, 0.2950796782970428, 0.11...","[[0.4557418823242187, 0.2926055192947387, 0.13...","[[0.4433253705501556, 0.2912991642951965, 0.13...",...,None,None,None,None,None,None,None,None,None,None
23,Punching,"[[0.3711828887462616, 0.2946740984916687, -0.0...","[[0.3712038695812225, 0.294670820236206, -0.03...","[[0.3723418414592743, 0.2949283123016357, 0.03...","[[0.3713643848896026, 0.2949852645397186, -0.0...","[[0.3686440885066986, 0.2955136597156524, -0.1...","[[0.3666049540042877, 0.2977917790412903, -0.1...","[[0.3667484223842621, 0.3022294044494629, -0.1...","[[0.3668553233146667, 0.3087800145149231, -0.2...","[[0.369797945022583, 0.3137444257736206, -0.18...",...,None,None,None,None,None,None,None,None,None,None
24,Punching,"[[0.3486809134483337, 0.2806042432785034, 0.03...","[[0.3468762934207916, 0.2841523289680481, 0.06...","[[0.3405708074569702, 0.2924183011054992, 0.11...","[[0.3342469930648803, 0.2982830703258514, 0.09...","[[0.3305109143257141, 0.3015491366386413, 0.08...","[[0.3282172977924347, 0.2991608083248138, 0.10...","[[0.3274014890193939, 0.297793447971344, 0.088...","[[0.3270829617977142, 0.2848131358623504, 0.08...","[[0.3273344933986664, 0.2753720879554748, -0.0...",...,None,None,None,None,None,None,None,None,None,None
25,Punching,"[[0.4130125343799591, 0.2571157813072204, -0.1...","[[0.4150344133377075, 0.2593288421630859, -0.2...","[[0.4143155515193939, 0.2617389857769012, -0.2...","[[0.4112043678760528, 0.2706581056118011, -0.2...","[[0.4102219939231872, 0.278572142124176, -0.21...","[[0.4102450609207153, 0.277853399515152, -0.20...","[[0.4112346172332763, 0.276483803987503, -0.22...","[[0.4154210090637207, 0.2751415371894836, -0.2...","[[0.4185889363288879, 0.2742774188518524, -0.0...",...,None,None,None,None,None,None,None,None,None,None
26,Punching,"[[0.460360050201416, 0.2338409423828125, -0.18...","[[0.4605927467346191, 0.2338456362485885, -0.1...","[[0.4587205350399017, 0.2337889969348907, -0.1...","[[0.456954687833786, 0.2332628071308136, -0.13...","[[0.457022100687027, 0.2332832515239715, -0.13...","[[0.457040786743164, 0.2335169017314911, -0.13...","[[0.4581725299358368, 0.2335172444581985, -0.2...","[[0.4587406516075134, 0.2349502742290496, -0.2...","[[0.4598631560802459, 0.2349376231431961, -0.2...",...,None,None,None,None,None,None,None,None,None,None
27,Punching,"[[0.4452933967113495, 0.2714939713478088, -0.1...","[[0.4452979266643524, 0.2714923024177551, -0.1...","[[0.4448969960212707, 0.2723944783210754, -0.1...","[[0.4447053670883178, 0.274724394083023, -0.12...","[[0.4446959197521209, 0.2751603722572326, -0.1...","[[0.4446808397769928, 0.2750848531723022, -0.1...","[[0.4437042772769928, 0.2751081585884094, -0.1...","[[0.44265016913414, 0.2750895619392395, -0.144...","[[0.4418310523033142, 0.275086760520935, -0.12...",...,None,None,None,None,None,None,None,None,None,None
28,Punching,"[[0.5175538659095764, 0.2564538717269897, -0.0...","[[0.5210102796554565, 0.2559376060962677, -0.0...","[[0.5209433436393738, 0.2558490037918091, -0.0...","[[0.5206713080406189, 0.2559